<a href="https://colab.research.google.com/github/cddogaru/Proyecto-de-Computacion/blob/main/An%C3%A1lisis_del_estado_y_evoluci%C3%B3n_de_los_embalses_de_agua_nacionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ANÁLISIS DEL ESTADO Y EVOLUCIÓN DE LOS EMBALSES DE AGUA NACIONALES**


Este notebook explica el trabajo de preprocesamiento, análisis y modelado para entender el estado y la evolución de la reserva hídrica en España. A lo largo del proyecto, se trabajará con datos históricos y geográficos para explorar tendencias, identificar patrones y realizar predicciones sobre el agua embalsada en el país.

Antes de empezar con los análisis y modelos, se seguirán los siguientes pasos:




# Pasos que se seguirán para preparar los datos
1. Importación de librerías y configuración inicial.
2. Carga de archivos de datos a utilizar.
3. Modificación y ajuste de las variables.
4. Detección y tratamiento de datos ausentes (NAs).
5. Generación de nuevas variables.
6. Enriquecimiento del dataset con datos climáticos y/o económicos.
7. Análisis exploratorio de los datos.
8. Modelado predictivo y comparativa de modelos.
9. Visualización de resultados.
10. Creación de tablas y exportación de datos.
11. Desarrollo de un prototipo interactivo.
12. Conclusiones y recomendaciones.





## 1. Importación de librerias
Lo primero que debemos hacer es instalar y cargar las librerías necesarias para el manejo, análisis y visualización de datos, además de configurar el entorno de trabajo. Usaremos algunas de las librerías más populares de Python para garantizar un flujo de trabajo eficiente.



In [18]:
try:
    # Importamos las librerías a utilizar
    import pandas as pd  # Manejo y análisis de datos
    import numpy as np  # Operaciones matemáticas
    import matplotlib.pyplot as plt  # Visualización básica
    import seaborn as sns  # Visualización estadística avanzada
    from sklearn.model_selection import train_test_split  # Dividir datos para modelado
    from sklearn.linear_model import LinearRegression  # Modelo de regresión lineal
    from statsmodels.tsa.arima.model import ARIMA  # Modelos ARIMA para series temporales

    # Configuramos el entorno
    import warnings
    warnings.filterwarnings("ignore")  # Ignoramos warnings para evitar distracciones

    # Configuración de pandas para mostrar más datos
    pd.set_option("display.max_columns", None)
    pd.set_option("display.max_rows", 100)

    # Configuramos el estilo de los gráficos
    print("Estilos disponibles en Matplotlib:", plt.style.available)
    plt.style.use("ggplot")  # Cambiamos a un estilo seguro como 'ggplot'
    plt.rcParams["figure.figsize"] = (10, 6)
    plt.rcParams["font.size"] = 12

    # Confirmación de configuración
    print("Librerías cargadas y entorno configurado correctamente.")

except ImportError as e:
    print(f"Error al importar librerías: {e}")
    print("Por favor, asegúrate de tener todas las librerías instaladas.")


Estilos disponibles en Matplotlib: ['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']
Librerías cargadas y entorno configurado correctamente.


## 2. Carga de datos

Para realizar el análisis y predicciones sobre el agua embalsada, es necesario cargar los datasets que contienen información histórica y geográfica de los embalses. Estos archivos están alojados en un repositorio de GitHub.

Archivos a cargar:
Datos embalses: Contiene datos históricos sobre el volumen de agua embalsada en diferentes fechas.
Embalses enriquecido: Contiene información geográfica y detalles adicionales sobre los embalses.


In [24]:
# Importamos pandas para manejar los datasets
import pandas as pd

# URLs de los archivos en el repositorio de GitHub
url_historicos = "https://raw.githubusercontent.com/cddogaru/Proyecto-de-Computaci-n-I/main/Datos%20embalses.xlsx"
url_geo = "https://raw.githubusercontent.com/cddogaru/Proyecto-de-Computaci-n-I/main/Embalses_enriquecido.xlsx"

# Cargar los datasets desde GitHub
try:
    # Cargar archivo histórico
    embalses = pd.read_excel(url_historicos)
    print("Datos embalses cargados con éxito.")

    # Cargar archivo geográfico
    embalses_geo = pd.read_excel(url_geo)
    print("Embalses enriquecido cargado con éxito.")
except Exception as e:
    print(f"Error al cargar los datasets: {e}")

# Resumen de los datos históricos
print("\nResumen de 'Datos embalses':")
print(embalses.info())
print("\nPrimeras filas de 'Datos embalses':")
print(embalses.head())

# Resumen de los datos geográficos
print("\nResumen de 'Embalses enriquecido':")
print(embalses_geo.info())
print("\nPrimeras filas de 'Embalses enriquecido':")
print(embalses_geo.head())

# --- Ajustes en Datos embalses ---
# Convertimos las columnas AGUA_TOTAL y AGUA_ACTUAL a formato numérico
embalses["AGUA_TOTAL"] = pd.to_numeric(embalses["AGUA_TOTAL"].str.replace(",", "."), errors="coerce")
embalses["AGUA_ACTUAL"] = pd.to_numeric(embalses["AGUA_ACTUAL"].str.replace(",", "."), errors="coerce")

# Eliminamos la columna ELECTRICO_FLAG si no es relevante
if "ELECTRICO_FLAG" in embalses.columns:
    embalses.drop(columns=["ELECTRICO_FLAG"], inplace=True)

# Verificamos los cambios en Datos embalses
print("\nDatos embalses tras ajustes:")
print(embalses.info())
print(embalses.head())

# --- Ajustes en Embalses enriquecido ---
# Limpiamos la columna USO para eliminar caracteres innecesarios
embalses_geo["USO"] = embalses_geo["USO"].str.replace("\n", " ").str.strip()

# Eliminamos columnas con demasiados valores nulos si no son críticas para el análisis
columnas_a_eliminar = ["Google Knowledge Graph ID", "Google", "Wikidata",
                       "OpenStreetMap relation ID", "OpenStreetMap", "image", "Imagen"]
embalses_geo.drop(columns=[col for col in columnas_a_eliminar if col in embalses_geo.columns], inplace=True)

# Verificamos los cambios en Embalses enriquecido
print("\nEmbalses enriquecido tras ajustes:")
print(embalses_geo.info())
print(embalses_geo.head())

Datos embalses cargados con éxito.
Embalses enriquecido cargado con éxito.

Resumen de 'Datos embalses':
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637606 entries, 0 to 637605
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   AMBITO_NOMBRE   637606 non-null  object        
 1   EMBALSE_NOMBRE  637606 non-null  object        
 2   FECHA           637606 non-null  datetime64[ns]
 3   AGUA_TOTAL      637604 non-null  object        
 4   AGUA_ACTUAL     637604 non-null  object        
 5   ELECTRICO_FLAG  637606 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 29.2+ MB
None

Primeras filas de 'Datos embalses':
  AMBITO_NOMBRE EMBALSE_NOMBRE      FECHA AGUA_TOTAL AGUA_ACTUAL  \
0    Miño - Sil     Albarellos 1988-01-05      91,00       32,00   
1    Miño - Sil     Albarellos 1988-01-12      91,00       44,00   
2    Miño - Sil     Albarellos 1988-01-19    

Estructura Documentada del Punto 2

1. Explicación Breve: Introducimos el propósito de cargar los datos y describimos los archivos utilizados.

2. Cargar Datos: Se usan las URLs de los archivos en GitHub para cargarlos directamente en el entorno Colab.

3. Validación:

  *   Se usa info() para mostrar un resumen de los datos (columnas, tipos,  
    valores nulos).
  *   Se usa head() para visualizar las primeras filas y asegurarnos de que los datos se cargaron correctamente.

## 3. Modificación y ajuste de las variables

Para realizar un análisis adecuado, es importante limpiar y ajustar las variables de los datasets cargados. Esto incluye renombrar columnas, ajustar los tipos de datos, eliminar columnas irrelevantes y crear nuevas variables cuando sea necesario.

*   Renombrar columnas: Los nombres de las columnas se ajustan para que sean más descriptivos y consistentes.

*   Ajustar tipos de datos: Se convierten columnas a formatos adecuados, como datetime para fechas y float para números.

*   Ajustar tipos de datos: Se convierten columnas a formatos adecuados, como datetime para fechas y float para números.

*   Eliminar columnas irrelevantes: Se eliminan aquellas columnas que no aportan valor al análisis.

*    Crear nuevas variables: Se combinan las coordenadas en una sola columna en el dataset geográfico.




**Ajuste del Dataset de Datos Históricos**

In [25]:
# Renombramos columnas para mayor claridad
columnas_a_renombrar = {
    "FECHA": "fecha",
    "AGUA_TOTAL": "capacidad_total",
    "AGUA_ACTUAL": "volumen_actual",
    "AMBITO_NOMBRE": "demarcacion_hidrografica"
}

# Verificamos si las columnas existen antes de renombrar para evitar errores
columnas_existentes = [col for col in columnas_a_renombrar.keys() if col in embalses.columns]
if columnas_existentes:
    embalses = embalses.rename(columns={col: columnas_a_renombrar[col] for col in columnas_existentes})
else:
    print("Advertencia: Ninguna de las columnas a renombrar se encontró en el dataset.")

# Convertimos el formato de fecha
try:
    embalses["fecha"] = pd.to_datetime(embalses["fecha"], errors="coerce")
except KeyError:
    print("Advertencia: La columna 'fecha' no se encuentra en el dataset.")

# Convertimos columnas numéricas al tipo float
for columna in ["capacidad_total", "volumen_actual"]:
    if columna in embalses.columns:
        embalses[columna] = pd.to_numeric(embalses[columna], errors="coerce")
    else:
        print(f"Advertencia: La columna '{columna}' no se encuentra en el dataset.")

# Eliminamos columnas innecesarias
if "ELECTRICO_FLAG" in embalses.columns:
    embalses = embalses.drop(columns=["ELECTRICO_FLAG"])
else:
    print("Advertencia: La columna 'ELECTRICO_FLAG' no se encuentra en el dataset.")

# Verificamos los cambios
print("\nDataset 'Datos embalses' tras ajustes:")
print(embalses.info())
print(embalses.head())

Advertencia: La columna 'ELECTRICO_FLAG' no se encuentra en el dataset.

Dataset 'Datos embalses' tras ajustes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637606 entries, 0 to 637605
Data columns (total 5 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   demarcacion_hidrografica  637606 non-null  object        
 1   EMBALSE_NOMBRE            637606 non-null  object        
 2   fecha                     637606 non-null  datetime64[ns]
 3   capacidad_total           637604 non-null  float64       
 4   volumen_actual            637604 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 24.3+ MB
None
  demarcacion_hidrografica EMBALSE_NOMBRE      fecha  capacidad_total  \
0               Miño - Sil     Albarellos 1988-01-05             91.0   
1               Miño - Sil     Albarellos 1988-01-12             91.0   
2               Miño - Sil     Albarellos

**Ajuste del Dataset Geográfico**

In [29]:
# Renombramos columnas para mayor claridad
columnas_a_renombrar = {
    "NOMBRE": "nombre_embalse",
    "X": "coordenada_x",
    "Y": "coordenada_y",
}
embalses_geo = embalses_geo.rename(columns=columnas_a_renombrar)

# Combinamos coordenadas en una nueva columna
embalses_geo["coordenadas"] = (
    embalses_geo["coordenada_x"].astype(str) + ", " + embalses_geo["coordenada_y"].astype(str)
)

# Limpieza de texto en columnas categóricas
columnas_a_limpiar = ["TITULAR", "CAUCE", "USUARIOS", "USO"]
for col in columnas_a_limpiar:
    embalses_geo[col] = embalses_geo[col].str.replace("\n", " ").str.strip()

# Reemplazo de valores nulos
embalses_geo["CAUCE"].fillna("Desconocido", inplace=True)
embalses_geo["USUARIOS"].fillna("No especificado", inplace=True)
columnas_numericas = ["SUP_CUENCA", "AP_M_ANUAL", "COTA_CORON", "ALT_CIMIEN"]
for col in columnas_numericas:
    embalses_geo[col].fillna(embalses_geo[col].median(), inplace=True)

# Verificamos los cambios
print("\nDataset 'Embalses enriquecido' tras ajustes:")
print(embalses_geo.info())
print(embalses_geo.head())


Dataset 'Embalses enriquecido' tras ajustes:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 367 entries, 0 to 366
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   coordenada_x    367 non-null    float64
 1   coordenada_y    367 non-null    float64
 2   INFORME         367 non-null    object 
 3   ID_INFRAES      367 non-null    int64  
 4   CODIGO          367 non-null    int64  
 5   nombre_embalse  367 non-null    object 
 6   EMBALSE         367 non-null    object 
 7   FASE            367 non-null    object 
 8   TITULAR         367 non-null    object 
 9   CATEGORIA       367 non-null    object 
 10  CAUCE           367 non-null    object 
 11  DEMARC          367 non-null    object 
 12  PROVINCIA       367 non-null    object 
 13  CCAA            367 non-null    object 
 14  TIPO            367 non-null    object 
 15  USUARIOS        367 non-null    object 
 16  USO             367 non-null    ob

**Resultados Esperados**
*   Dataset de datos históricos: Los nombres de las columnas serán más claros, las fechas estarán en formato datetime, y los valores numéricos estarán en formato float.

*   Dataset geográfico: Tendremos coordenadas combinadas en una nueva columna, y se eliminarán columnas irrelevantes.

## **4. Detección y Tratamiento de Datos Ausentes (NAs)**

En esta etapa, identificamos y tratamos los valores ausentes en ambos datasets para garantizar que los datos estén completos y listos para el análisis. Los pasos realizados incluyen:

1. **Identificar valores ausentes:** Verificamos la cantidad de valores nulos en cada columna.
2. **Tratamiento de valores ausentes:**
   - Se eliminan filas con valores nulos en columnas críticas.
   - Se rellenan valores faltantes en columnas menos críticas con valores predeterminados.

### **Detección de Valores Ausentes**

In [31]:
# Tratamiento de valores ausentes en 'Datos embalses'
embalses["capacidad_total"].fillna(embalses["capacidad_total"].mean(), inplace=True)
embalses["volumen_actual"].fillna(embalses["volumen_actual"].mean(), inplace=True)

# Tratamiento de valores ausentes en 'Embalses enriquecido'
embalses_geo["LONG_CORON"].fillna(embalses_geo["LONG_CORON"].median(), inplace=True)
embalses_geo["CAP_AL_NAE"].fillna(embalses_geo["CAP_AL_NAE"].median(), inplace=True)
embalses_geo["CAP_MAX_DF"].fillna(embalses_geo["CAP_MAX_DF"].median(), inplace=True)
embalses_geo["NMN_SUP"].fillna(embalses_geo["NMN_SUP"].median(), inplace=True)

# Verificamos los valores nulos tras el tratamiento
print("Valores nulos tras tratamiento en 'Datos embalses':")
print(embalses.isnull().sum())

print("\nValores nulos tras tratamiento en 'Embalses enriquecido':")
print(embalses_geo.isnull().sum())

Valores nulos tras tratamiento en 'Datos embalses':
demarcacion_hidrografica    0
EMBALSE_NOMBRE              0
fecha                       0
capacidad_total             0
volumen_actual              0
dtype: int64

Valores nulos tras tratamiento en 'Embalses enriquecido':
coordenada_x      0
coordenada_y      0
INFORME           0
ID_INFRAES        0
CODIGO            0
nombre_embalse    0
EMBALSE           0
FASE              0
TITULAR           0
CATEGORIA         0
CAUCE             0
DEMARC            0
PROVINCIA         0
CCAA              0
TIPO              0
USUARIOS          0
USO               0
SUP_CUENCA        0
AP_M_ANUAL        0
NMN_CAPAC         0
NMN_SUP           0
COTA_CORON        0
ALT_CIMIEN        0
LONG_CORON        0
CAP_AL_NAE        0
CAP_MAX_DF        0
coordenadas       0
dtype: int64


**Tratamiento de Valores Ausentes**

Dataset Histórico

In [32]:
# Tratamiento de valores ausentes en 'Datos embalses'
print("Tratamiento de valores ausentes en 'Datos embalses':")

# Verificar si el dataset está vacío antes de continuar
if embalses.empty:
    print("Advertencia: El dataset 'Datos embalses' está vacío antes del tratamiento.")
else:
    # Imputamos valores faltantes en las columnas críticas con la media
    embalses["capacidad_total"].fillna(embalses["capacidad_total"].mean(), inplace=True)
    embalses["volumen_actual"].fillna(embalses["volumen_actual"].mean(), inplace=True)

    # Verificamos nuevamente
    print("\nValores nulos tras imputación en 'Datos embalses':")
    print(embalses.isnull().sum())

    # Mostrar las primeras filas del dataset
    print("\nPrimeras filas del Dataset Histórico tras el tratamiento:")
    print(embalses.head())

# Verificar si el dataset sigue vacío después del tratamiento
if embalses.empty:
    print("Advertencia: El dataset 'Datos embalses' sigue vacío tras el tratamiento.")

Tratamiento de valores ausentes en 'Datos embalses':

Valores nulos tras imputación en 'Datos embalses':
demarcacion_hidrografica    0
EMBALSE_NOMBRE              0
fecha                       0
capacidad_total             0
volumen_actual              0
dtype: int64

Primeras filas del Dataset Histórico tras el tratamiento:
  demarcacion_hidrografica EMBALSE_NOMBRE      fecha  capacidad_total  \
0               Miño - Sil     Albarellos 1988-01-05             91.0   
1               Miño - Sil     Albarellos 1988-01-12             91.0   
2               Miño - Sil     Albarellos 1988-01-19             91.0   
3               Miño - Sil     Albarellos 1988-01-26             91.0   
4               Miño - Sil     Albarellos 1988-02-02             91.0   

   volumen_actual  
0            32.0  
1            44.0  
2            42.0  
3            43.0  
4            65.0  


Dataset Geográfico

In [34]:
print("\nTratamiento de valores ausentes en 'Embalses enriquecido':")

# Eliminamos filas con valores nulos en columnas críticas
embalses_geo = embalses_geo.dropna(subset=["nombre_embalse", "coordenada_x", "coordenada_y"])

# Rellenamos valores faltantes en columnas menos críticas con valores predeterminados
embalses_geo["DEMARC"] = embalses_geo["DEMARC"].fillna("Desconocido")
embalses_geo["USO"] = embalses_geo["USO"].fillna("Sin especificar")

# Verificamos nuevamente
print("\nValores nulos tras el tratamiento en 'Embalses enriquecido':")
print(embalses_geo.isnull().sum())
print("\nPrimeras filas del Dataset Geográfico tras el tratamiento:")
print(embalses_geo.head())


Tratamiento de valores ausentes en 'Embalses enriquecido':

Valores nulos tras el tratamiento en 'Embalses enriquecido':
coordenada_x      0
coordenada_y      0
INFORME           0
ID_INFRAES        0
CODIGO            0
nombre_embalse    0
EMBALSE           0
FASE              0
TITULAR           0
CATEGORIA         0
CAUCE             0
DEMARC            0
PROVINCIA         0
CCAA              0
TIPO              0
USUARIOS          0
USO               0
SUP_CUENCA        0
AP_M_ANUAL        0
NMN_CAPAC         0
NMN_SUP           0
COTA_CORON        0
ALT_CIMIEN        0
LONG_CORON        0
CAP_AL_NAE        0
CAP_MAX_DF        0
coordenadas       0
dtype: int64

Primeras filas del Dataset Geográfico tras el tratamiento:
   coordenada_x  coordenada_y  \
0     36.153615     -5.649731   
1     42.722873     -0.462684   
2     38.095562     -3.831098   
3     43.277118     -1.773747   
4     38.060263     -1.487653   

                                             INFORME  ID_INFRAES  

**Resultados**
1. Los valores ausentes en columnas críticas han sido eliminados.
2. Las columnas no críticas han sido rellenadas con valores predeterminados.

## **5. Generación de Nuevas Variables**

En esta sección, enriquecemos los datasets con nuevas variables derivadas de los datos existentes para facilitar análisis más profundos. También validamos que los datasets estén en buen estado antes de realizar los cálculos.

### **Validación del Estado de los Datasets**

Antes de generar nuevas variables, verificamos si los datasets están vacíos tras las operaciones previas de limpieza.

In [17]:
# Verificamos si el dataset está vacío
if embalses.empty:
    print("Advertencia: El dataset 'embalses' está vacío después de la limpieza.")
else:
    print("Dataset 'embalses' listo para generar nuevas variables.")

if embalses_geo.empty:
    print("Advertencia: El dataset 'embalses_geo' está vacío después de la limpieza.")
else:
    print("Dataset 'embalses_geo' listo para generar nuevas variables.")

Advertencia: El dataset 'embalses' está vacío después de la limpieza.
Dataset 'embalses_geo' listo para generar nuevas variables.


### **Nuevas Variables Creadas**

#### **Dataset Histórico (embalses)**
1. **porcentaje_actual:** Porcentaje de llenado del embalse, calculado como (volumen_actual / capacidad_total) * 100.
2. **año y mes:** Año y mes extraídos de la columna fecha.


In [13]:
# Generamos nuevas variables en el Dataset Histórico
embalses["porcentaje_actual"] = (embalses["volumen_actual"] / embalses["capacidad_total"]) * 100
embalses["año"] = embalses["fecha"].dt.year
embalses["mes"] = embalses["fecha"].dt.month

# Verificamos si hay valores nulos en las nuevas variables
print("Valores nulos en las nuevas variables del Dataset Histórico:")
print(embalses[["porcentaje_actual", "año", "mes"]].isnull().sum())

# Mostramos las primeras filas para verificar
print("\nDataset 'Datos embalses' tras generación de nuevas variables:")
print(embalses.head())


Valores nulos en las nuevas variables del Dataset Histórico:
porcentaje_actual    0
año                  0
mes                  0
dtype: int64

Dataset 'Datos embalses' tras generación de nuevas variables:
Empty DataFrame
Columns: [demarcacion_hidrografica, EMBALSE_NOMBRE, fecha, capacidad_total, volumen_actual, porcentaje_actual, año, mes]
Index: []


**Dataset Geográfico (embalses_geo)**
1. tipo_uso_limpio: Columna USO limpia, con caracteres innecesarios removidos.
2. capacidad_total_calculada: Capacidad total calculada como la media entre NMN_CAPAC y CAP_MAX_DF.
3. (Opcional) categoria_capacidad: Clasifica los embalses en "Pequeño", "Mediano" o "Grande" según la capacidad total calculada.

In [14]:
# Limpiamos la columna "USO" para crear una versión más legible
embalses_geo["tipo_uso_limpio"] = embalses_geo["USO"].str.replace("\n", " ").str.strip()

# Generamos una nueva columna "capacidad_total_calculada" basada en columnas existentes
embalses_geo["capacidad_total_calculada"] = embalses_geo[["NMN_CAPAC", "CAP_MAX_DF"]].mean(axis=1)

# Clasificación opcional de embalses por capacidad
def clasificar_capacidad(capacidad):
    if capacidad < 100:
        return "Pequeño"
    elif capacidad < 500:
        return "Mediano"
    else:
        return "Grande"

embalses_geo["categoria_capacidad"] = embalses_geo["capacidad_total_calculada"].apply(clasificar_capacidad)

# Verificamos si hay valores nulos en las nuevas variables
print("Valores nulos en las nuevas variables del Dataset Geográfico:")
print(embalses_geo[["tipo_uso_limpio", "capacidad_total_calculada", "categoria_capacidad"]].isnull().sum())

# Mostramos las primeras filas para verificar
print("\nDataset 'Embalses enriquecido' tras generación de nuevas variables:")
print(embalses_geo.head())


Valores nulos en las nuevas variables del Dataset Geográfico:
tipo_uso_limpio              0
capacidad_total_calculada    0
categoria_capacidad          0
dtype: int64

Dataset 'Embalses enriquecido' tras generación de nuevas variables:
   coordenada_x  coordenada_y  \
0     36.153615     -5.649731   
1     42.722873     -0.462684   
2     38.095562     -3.831098   
3     43.277118     -1.773747   
4     38.060263     -1.487653   

                                             INFORME  ID_INFRAES   CODIGO  \
0  https://sig.mapama.gob.es/WebServices/clientew...        1364  5110039   
1  https://sig.mapama.gob.es/WebServices/clientew...        2482  9220067   
2  https://sig.mapama.gob.es/WebServices/clientew...        1543  5230007   
3  https://sig.mapama.gob.es/WebServices/clientew...          84  1310009   
4  https://sig.mapama.gob.es/WebServices/clientew...        2057  7300021   

  nombre_embalse               EMBALSE Google Knowledge Graph ID  \
0      ALMODOVAR  Embalse de Almo

**Resultados**
1. Dataset Histórico: Incluye las nuevas variables porcentaje_actual, año, y mes.
2. Dataset Geográfico: Incluye las nuevas variables tipo_uso_limpio y capacidad_total_calculada.

## 6. Creación de tabla para visualización "Evolución histórica de la reserva hídrica entre los años 2012 y 2022"
Generamos la primera tabla de datos preparada para alimentar la herramienta de visualización que vamos a utilizar, Google Data Studio.

Para ello **filtramos** el dataset para obtener los datos entre el **01/01/2012 y el 01/01/2022**






In [ ]:
# Filtramos el dataset para quedarnos con los valores históricos del 2012 al 2022
tabla_lin = embalses[(embalses["fecha"]>"01/01/2012") & (embalses["fecha"]<"01/01/2022")]
tabla_lin.head()

Visualizamos las 5 primeras filas de la tabla y comprobamos que se ha **filtrado** correctamente.

## 7. Creación de tabla para visualización "Reserva hídrica (hm3) entre los años 2012 y 2022"

Generamos **una nueva tabla de datos** preparada para alimentar la herramienta de visualización que vamos a utilizar, Google Data Studio.

Para ello partimos del **dataset filtrado** en el apartado anterior. **Agrupamos** las variables por embalse, **calculamos la media** de los registros del volumen de agua y **renombramos** la nueva variable.


*   Para agrupar las varibles utilizamos la función [.groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

*   Para renombrar las variables utilizamos las funciones [.rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)







In [ ]:
# Agrupamos el dataset por nombre del embalse y generamos una columna con la media del volumen
tabla_vol = round(tabla_lin.groupby(["demarcacion_hidrografica", "embalse_nombre"])["volumen_actual"].mean().reset_index(),2)

# Renombramos la nueva columna generada
tabla_vol = tabla_vol.rename(columns= {"volumen_actual": "volumen_medio_10"})
tabla_vol.head()


Una vez generada la tabla **"tabla_vol"** la visualizamos para comprobar que los datos son correctos. En este caso se comprueba que se ha calculado el valor medio de volumen de agua por embalse y ha sido almacenada en una nueva variable "**"volumen_medio_10"**


## 8. Creación de tabla para visualización "Reserva hídrica (%) entre los años 2012 y 2022"

Generamos una **nueva tabla de datos** preparada para alimentar la herramienta de visualización que vamos a utilizar, Google Data Studio.

Para ello partimos del **dataset filtrado** en el apartado anterior. **Agrupamos** las variables por embalse, **calculamos la media** de los registros del porcentaje de llenado y **renombramos** la nueva variable.


*   Para agrupar las varibles utilizamos la función [.groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

*   Para renombrar las variables utilizamos las funciones [.rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)



In [ ]:
# Agrupamos el dataset por nombre del embalse y generamos una columna con la media del porcentaje
tabla_por = round(tabla_lin.groupby(["demarcacion_hidrografica", "embalse_nombre"])["porcentaje_actual"].mean().reset_index(),2)

# Renombramos la nueva columna generada
tabla_por = tabla_por.rename(columns= {"porcentaje_actual": "porcentaje_medio_10"})
tabla_por.head()

Una vez generada la tabla **"tabla_por"** la visualizamos para comprobar que los datos son correctos. En este caso se comprueba que ha sido calculado el valor medio de porcentaje de llenado por embalse y ha sido almacenado en una nueva variable "**"porcentaje_medio_10"**

## 9. Creación de tabla para visualización "Evolución mensual de la reserva hídrica (hm3)"

Generamos una **nueva tabla de datos** preparada para alimentar la herramienta de visualización que vamos a utilizar, **Google Data Studio**.

Esta tabla contiene los datos medios de volumen de agua embalsada por mes para distintas series temporales. Esta series temporales corresponden a los años **2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020 y 2021**.

Para ello partimos del **dataset filtrado en el apartado 5** con los datos pertenecientes al periodo temporal entre el 01/01/2012 y el 01/01/2022. Una vez filtrado, **agrupamos** las variables por embalse y mes, **calculamos la media** de los registros del volumen de agua y **renombramos** la nueva variable.


*   Para agrupar las varibles utilizamos la función [groupby()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html)

*   Para renombrar las variables utilizamos la funciones [.rename()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)

*   Para unir las tablas en utilizamos la función [.merge()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html)

In [ ]:
# Creamos una lista con las fechas iniciales y finales de las series temporales
lista = ["01/01/2012", "01/01/2013", "01/01/2014", "01/01/2015", "01/01/2016", "01/01/2017", "01/01/2018", "01/01/2019", "01/01/2020", "01/01/2021", "01/01/2022"]

# Creamos un bucle que recorra la lista anterior
for i in range(len(lista)-1):
    # Filtramos el dataset con los datos correspondientes a cada una de las series temporales
    tabla_mini = tabla_lin[(embalses["fecha"] > lista[i]) & (embalses["fecha"]<lista[i+1])]
    # Agrupamos el dataset por demarcación hidrográfica, nombre del embalse y mes. Generamos una columna con la media del volumen
    lin = tabla_mini.groupby(["demarcacion_hidrografica","embalse_nombre", "mes"])["volumen_actual"].mean().reset_index()
    # Renombramos la nueva columna generada con el valor medio
    lin = lin.rename(columns= {"volumen_actual": lista[i].split("/")[2]})

    # Unificamos los datos de las distintas series temporales en una misma tabla
    if i == 0:
      tabla_lin_mes = lin
    else:
      tabla_lin_mes = tabla_lin_mes.merge(lin, on=["demarcacion_hidrografica","embalse_nombre","mes"], how = "inner")

tabla_lin_mes.head()

Una vez generada la tabla **"tabla_lin_mes"** la visualizamos para comprobar que los datos son correctos. En este caso se comprueba que los datos del "volumen_actual" han sido agrupados por **mes** y en las **distintas series temporales** previamente especificadas.

## 10. Guardado de las tablas para la generación de la visualización.

Una vez que tenemos las tablas con la estructura y variables que nos interesan para realizar la visualización de los datos, lo guardaremos como archivo de datos en formato **CSV** para posteriormente realizar otros análisis estadísticos o utilizarlo en otras herramientas de visualización de datos como la que abordamos a continuación. Es importante guardarlo con una codificación **UTF-8** (Formato de Transformación Unicode) para que los caracteres especiales sean identificados de manera correcta por cualquier software.

*   Para guardar las tablas como archivos CSV utilizamos la función [.to_csv()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html)




In [ ]:
# Guardamos las tablas como archivos csv.
tabla_lin.to_csv("lineas.csv", index = False, encoding = "utf-8")
tabla_vol.to_csv("volumen.csv", index = False, encoding = "utf-8")
tabla_por.to_csv("porcentaje.csv", index = False, encoding = "utf-8")
tabla_lin_mes.to_csv("lineas_mensual.csv", index = False, encoding = "utf-8")
embalses_geo.to_csv("geo.csv", index = False, encoding = "utf-8")





Una vez generados los archivos, en el menú desplegable de la izquierda de esta pantalla, en la sección "Archivos" (el icono de la carpeta), encontraremos los archivos que acabamos de guardar dentro de la carpeta "sample_data". Usando el menú contextual, los ficheros pueden ser descargados.

No obstante, dispones de estos conjuntos de datos preprocesados en esta carpeta del Laboratorio de datos del GitHub de datos.gob.es.

A contintuación, puedes seguir los siguientes pasos para visualizar los datos que acabamos de preprocesar utilizando la herramienta Google Data Studio en el post *(añadir nombre y enlace al post)*